In [7]:
from operator import itemgetter

d = {"a": ["b"], "c": ["d"]}
itemgetter(*["a", "c"])(d)

(['b'], ['d'])

In [1]:
import pickle
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
from datasets import load_dataset
from torch.utils.data import DataLoader

/home/luigi/anaconda3/envs/chocolm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = torch.device('cuda')

chord_tokenizer = AutoTokenizer.from_pretrained("jammai/chocolm-modernbert-base")
chord_model = AutoModel.from_pretrained("jammai/chocolm-modernbert-base")
chord_model.to(device)

#text_tokenizer = AutoTokenizer.from_pretrained("neavo/modern_bert_multilingual")
#text_model = AutoModel.from_pretrained("neavo/modern_bert_multilingual")
text_tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
text_model = AutoModel.from_pretrained("xlm-roberta-base", attn_implementation="eager")
text_model.to(device)

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [ ]:
torch.no_grad()
tokenized = text_tokenizer(["abc", "cde efg"], return_tensors="pt", padding=True)
tokenized = tokenized.to(device)
model_output = text_model(**tokenized, output_hidden_states=True)

In [ ]:
tokenized.attention_mask

tensor([[1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1]], device='cuda:0')

In [ ]:
model_output.last_hidden_state.size(), tokenized.attention_mask.size()

(torch.Size([2, 4, 768]), torch.Size([2, 4]))

In [34]:
ms = torch.Tensor([[[[1,2,3,4],[5,6,7,8],[8,1,1,3]],[[1,2,3,4],[6,7,8,10],[7,8,9,1]]],
                  [[[2,3,4,5],[6,6,7,9],[8,1,4,3]],[[1,2,4,4],[6,7,8,11],[7,8,1,1]]]])
ms.size()

torch.Size([2, 2, 3, 4])

In [2]:
a = torch.Tensor([[0,1,1],[1,0,1]])
a.size()

torch.Size([2, 3])

In [4]:
a

tensor([[0., 1., 1.],
        [1., 0., 1.]])

In [ ]:
a = a.unsqueeze(0).repeat(2,1,1)

In [ ]:
a@ms

tensor([[[[13.,  7.,  8., 11.],
          [ 9.,  3.,  4.,  7.]],

         [[13., 15., 17., 11.],
          [ 8., 10., 12.,  5.]]],


        [[[14.,  7., 11., 12.],
          [10.,  4.,  8.,  8.]],

         [[13., 15.,  9., 12.],
          [ 8., 10.,  5.,  5.]]]])

In [52]:
(a@ms)[0][0][0]

tensor([13.,  7.,  8., 11.])

In [53]:
(a@ms)[0][1][1]

tensor([ 8., 10., 12.,  5.])

In [ ]:
torch.vstack([torch.vstack([(a@ms)[layer][i][i] for layer in range(2)  ]) for i in range(2) ])

tensor([[13.,  7.,  8., 11.],
        [ 8., 10., 12.,  5.],
        [14.,  7., 11., 12.],
        [ 8., 10.,  5.,  5.]])